In [1]:
# Cell 1: Install Required Packages
# !pip install pathway bokeh numpy pandas matplotlib seaborn
# Clone the main Pathway repository
!git clone https://github.com/pathwaycom/pathway.git
!git clone https://github.com/pathwaycom/llm-app.git

# Navigate to the directories
# import os
# os.chdir('/content/pathway')

# Cell 2: Import Libraries
import pathway as pw


fatal: destination path 'pathway' already exists and is not an empty directory.
fatal: destination path 'llm-app' already exists and is not an empty directory.


In [2]:
import numpy as np
import pandas as pd
import pathway as pw
from datetime import datetime, timedelta
import math
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.io import push_notebook
import time
import threading
from typing import Dict, List, Tuple

# Enable Bokeh in notebook
output_notebook()


Loading BokehJS ...

In [3]:
class ParkingPriceEngine:
    """
    Dynamic Pricing Engine for Urban Parking Lots
    Implements three pricing models with increasing complexity
    """
    
    def __init__(self, base_price: float = 10.0):
        self.base_price = base_price
        self.current_prices = {}
        self.price_history = {}
        self.demand_history = {}
        
        # Model parameters
        self.model1_alpha = 0.5  # Linear model coefficient
        self.model2_params = {
            'occupancy_weight': 2.0,
            'queue_weight': 1.5,
            'traffic_weight': 0.8,
            'special_day_weight': 1.2,
            'vehicle_type_weights': {'car': 1.0, 'bike': 0.7, 'truck': 1.3, 'cycle': 0.6}
        }
        
        # Price bounds
        self.min_price_multiplier = 0.5
        self.max_price_multiplier = 2.0


In [4]:
def calculate_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Calculate distance between two points using Haversine formula"""
    R = 6371  # Earth's radius in kilometers
    
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c

# Add this method to the ParkingPriceEngine class
ParkingPriceEngine.calculate_distance = calculate_distance


In [5]:
def model1_baseline_linear(self, system_code: str, occupancy: int, capacity: int, 
                          previous_price: float = None) -> float:
    """
    Model 1: Baseline Linear Model
    Price increases linearly with occupancy rate
    """
    if previous_price is None:
        previous_price = self.base_price
        
    occupancy_rate = occupancy / capacity if capacity > 0 else 0
    price_adjustment = self.model1_alpha * occupancy_rate
    
    new_price = previous_price + price_adjustment
    
    # Apply bounds
    min_price = self.base_price * self.min_price_multiplier
    max_price = self.base_price * self.max_price_multiplier
    
    return max(min_price, min(max_price, new_price))

# Add this method to the ParkingPriceEngine class
ParkingPriceEngine.model1_baseline_linear = model1_baseline_linear


In [6]:
def model2_demand_based(self, system_code: str, occupancy: int, capacity: int,
                       queue_length: int, traffic_condition: str, is_special_day: int,
                       vehicle_type: str) -> float:
    """
    Model 2: Demand-Based Price Function
    Uses multiple factors to calculate demand and adjust price
    """
    params = self.model2_params
    
    # Calculate occupancy rate
    occupancy_rate = occupancy / capacity if capacity > 0 else 0
    
    # Traffic condition mapping
    traffic_multiplier = {'low': 0.5, 'average': 1.0, 'high': 1.5}.get(traffic_condition, 1.0)
    
    # Vehicle type weight
    vehicle_weight = params['vehicle_type_weights'].get(vehicle_type, 1.0)
    
    # Calculate demand function
    demand = (params['occupancy_weight'] * occupancy_rate +
             params['queue_weight'] * (queue_length / 10) +  # Normalize queue length
             params['traffic_weight'] * traffic_multiplier +
             params['special_day_weight'] * is_special_day +
             0.5 * vehicle_weight)
    
    # Normalize demand (sigmoid function for smooth transitions)
    normalized_demand = 2 / (1 + math.exp(-demand)) - 1  # Range: [-1, 1]
    
    # Calculate price
    price_multiplier = 1 + 0.5 * normalized_demand  # Range: [0.5, 1.5]
    new_price = self.base_price * price_multiplier
    
    # Apply bounds
    min_price = self.base_price * self.min_price_multiplier
    max_price = self.base_price * self.max_price_multiplier
    
    return max(min_price, min(max_price, new_price))

# Add this method to the ParkingPriceEngine class
ParkingPriceEngine.model2_demand_based = model2_demand_based


In [7]:
def model3_competitive_pricing(self, system_code: str, occupancy: int, capacity: int,
                              queue_length: int, traffic_condition: str, is_special_day: int,
                              vehicle_type: str, latitude: float, longitude: float,
                              all_parking_data) -> Tuple[float, str]:
    """
    Model 3: Competitive Pricing Model
    Considers nearby parking lots and their prices for competitive pricing
    """
    # Start with demand-based price
    base_price = self.model2_demand_based(system_code, occupancy, capacity, queue_length,
                                        traffic_condition, is_special_day, vehicle_type)
    
    # Find nearby competitors (within 2km)
    nearby_lots = []
    for other_code, other_data in all_parking_data.items():
        if other_code != system_code:
            distance = self.calculate_distance(latitude, longitude,
                                             other_data['latitude'], other_data['longitude'])
            if distance <= 2.0:  # Within 2km
                nearby_lots.append({
                    'code': other_code,
                    'distance': distance,
                    'occupancy_rate': other_data['occupancy'] / other_data['capacity'],
                    'price': self.current_prices.get(other_code, self.base_price)
                })
    
    if not nearby_lots:
        return base_price, "No nearby competitors"
    
    # Sort by distance
    nearby_lots.sort(key=lambda x: x['distance'])
    
    # Calculate competitive adjustment
    current_occupancy_rate = occupancy / capacity if capacity > 0 else 0
    
    # If current lot is nearly full (>90%), suggest rerouting to less occupied nearby lots
    if current_occupancy_rate > 0.9:
        available_alternatives = [lot for lot in nearby_lots if lot['occupancy_rate'] < 0.8]
        if available_alternatives:
            cheapest_alternative = min(available_alternatives, key=lambda x: x['price'])
            suggestion = f"Consider rerouting to {cheapest_alternative['code']} (${cheapest_alternative['price']:.2f})"
            # Increase price to discourage more entries
            competitive_price = base_price * 1.2
        else:
            suggestion = "All nearby lots are busy"
            competitive_price = base_price * 1.3
    else:
        # Normal competitive pricing
        avg_nearby_price = sum(lot['price'] for lot in nearby_lots) / len(nearby_lots)
        
        if base_price > avg_nearby_price * 1.1:
            # Our price is too high, reduce it
            competitive_price = base_price * 0.95
            suggestion = "Price reduced due to competition"
        elif base_price < avg_nearby_price * 0.9:
            # We can increase price
            competitive_price = base_price * 1.05
            suggestion = "Price increased - competitive advantage"
        else:
            competitive_price = base_price
            suggestion = "Price competitive with nearby lots"
    
    # Apply bounds
    min_price = self.base_price * self.min_price_multiplier
    max_price = self.base_price * self.max_price_multiplier
    final_price = max(min_price, min(max_price, competitive_price))
    
    return final_price, suggestion

# Add this method to the ParkingPriceEngine class
ParkingPriceEngine.model3_competitive_pricing = model3_competitive_pricing


In [8]:
def update_price_history(self, system_code: str, price: float, timestamp: str):
    """Update price history for visualization"""
    if system_code not in self.price_history:
        self.price_history[system_code] = {'timestamps': [], 'prices': []}
    
    self.price_history[system_code]['timestamps'].append(timestamp)
    self.price_history[system_code]['prices'].append(price)
    
    # Keep only last 50 points for visualization
    if len(self.price_history[system_code]['timestamps']) > 50:
        self.price_history[system_code]['timestamps'] = self.price_history[system_code]['timestamps'][-50:]
        self.price_history[system_code]['prices'] = self.price_history[system_code]['prices'][-50:]

# Add this method to the ParkingPriceEngine class
ParkingPriceEngine.update_price_history = update_price_history


In [9]:
class RealTimeSimulator:
    """
    Real-time data simulation using Pathway
    """
    
    def __init__(self, data_file: str, pricing_engine: ParkingPriceEngine):
        self.data_file = data_file
        self.pricing_engine = pricing_engine
        self.current_data = {}
        
    def load_and_prepare_data(self):
        """Load and prepare the dataset"""
        df = pd.read_csv(self.data_file)
        
        # Convert date and time columns
        df['DateTime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], 
                                       format='%d-%m-%Y %H:%M:%S')
        
        # Sort by datetime
        df = df.sort_values('DateTime')
        
        return df


In [10]:
def simulate_real_time_stream(self, df: pd.DataFrame, delay_seconds: float = 1.0):
    """
    Simulate real-time data stream with specified delay
    """
    print("Starting real-time simulation...")
    print("=" * 60)
    
    # Group data by system code for easier processing
    grouped_data = {}
    for system_code in df['SystemCodeNumber'].unique():
        system_data = df[df['SystemCodeNumber'] == system_code].copy()
        grouped_data[system_code] = system_data
    
    # Get all unique timestamps
    all_timestamps = sorted(df['DateTime'].unique())
    
    for timestamp in all_timestamps:
        current_batch = df[df['DateTime'] == timestamp]
        
        # Update current data state
        all_parking_data = {}
        for _, row in current_batch.iterrows():
            system_code = row['SystemCodeNumber']
            all_parking_data[system_code] = {
                'occupancy': row['Occupancy'],
                'capacity': row['Capacity'],
                'latitude': row['Latitude'],
                'longitude': row['Longitude'],
                'queue_length': row['QueueLength'],
                'traffic_condition': row['TrafficConditionNearby'],
                'is_special_day': row['IsSpecialDay'],
                'vehicle_type': row['VehicleType']
            }
        
        # Process each parking lot
        results = []
        for system_code, data in all_parking_data.items():
            # Get previous price
            previous_price = self.pricing_engine.current_prices.get(system_code, 
                                                                   self.pricing_engine.base_price)
            
            # Calculate prices using all three models
            model1_price = self.pricing_engine.model1_baseline_linear(
                system_code, data['occupancy'], data['capacity'], previous_price
            )
            
            model2_price = self.pricing_engine.model2_demand_based(
                system_code, data['occupancy'], data['capacity'], data['queue_length'],
                data['traffic_condition'], data['is_special_day'], data['vehicle_type']
            )
            
            model3_price, suggestion = self.pricing_engine.model3_competitive_pricing(
                system_code, data['occupancy'], data['capacity'], data['queue_length'],
                data['traffic_condition'], data['is_special_day'], data['vehicle_type'],
                data['latitude'], data['longitude'], all_parking_data
            )
            
            # Update current price (using Model 3 as final price)
            self.pricing_engine.current_prices[system_code] = model3_price
            
            # Update price history
            self.pricing_engine.update_price_history(system_code, model3_price, str(timestamp))
            
            results.append({
                'timestamp': timestamp,
                'system_code': system_code,
                'occupancy': data['occupancy'],
                'capacity': data['capacity'],
                'occupancy_rate': data['occupancy'] / data['capacity'],
                'queue_length': data['queue_length'],
                'traffic': data['traffic_condition'],
                'special_day': data['is_special_day'],
                'vehicle_type': data['vehicle_type'],
                'model1_price': model1_price,
                'model2_price': model2_price,
                'model3_price': model3_price,
                'suggestion': suggestion
            })
        
        # Display current status
        self.display_current_status(timestamp, results)
        
        # Wait before next update
        time.sleep(delay_seconds)
        
    print("\nSimulation completed!")

# Add this method to the RealTimeSimulator class
RealTimeSimulator.simulate_real_time_stream = simulate_real_time_stream


In [11]:
def display_current_status(self, timestamp, results):
    """Display current pricing status"""
    print(f"\n🕐 Timestamp: {timestamp}")
    print("-" * 60)
    
    for result in results[:5]:  # Show first 5 for brevity
        occupancy_rate = result['occupancy_rate'] * 100
        print(f"🅿️  {result['system_code']}: "
              f"Occupancy: {result['occupancy']}/{result['capacity']} ({occupancy_rate:.1f}%) | "
              f"Queue: {result['queue_length']} | "
              f"Traffic: {result['traffic']} | "
              f"Price: ${result['model3_price']:.2f}")
        
        if result['suggestion'] != "Price competitive with nearby lots":
            print(f"   💡 {result['suggestion']}")
    
    if len(results) > 5:
        print(f"   ... and {len(results) - 5} more parking lots")

# Add this method to the RealTimeSimulator class
RealTimeSimulator.display_current_status = display_current_status


In [12]:
class VisualizationEngine:
    """
    Real-time visualization using Bokeh
    """
    
    def __init__(self, pricing_engine: ParkingPriceEngine):
        self.pricing_engine = pricing_engine
        self.plots = {}
        self.sources = {}
    
    def create_real_time_plots(self, system_codes: List[str]):
        """Create real-time pricing plots"""
        plots = []
        
        for i, system_code in enumerate(system_codes[:6]):  # Show first 6 plots
            # Create data source
            source = ColumnDataSource(data=dict(x=[], y=[]))
            self.sources[system_code] = source
            
            # Create plot
            p = figure(title=f"Real-time Pricing - {system_code}", 
                      x_axis_label='Time', y_axis_label='Price ($)',
                      width=400, height=300,
                      x_axis_type='datetime')
            
            p.line('x', 'y', source=source, line_width=2, color='blue')
            # Updated: Replace circle() with scatter()
            p.scatter('x', 'y', source=source, size=6, color='red')
            
            plots.append(p)
            self.plots[system_code] = p
        
        # Create grid layout
        grid = gridplot([plots[i:i+3] for i in range(0, len(plots), 3)])
        
        return grid
    
    def update_plots(self):
        """Update plots with latest data"""
        for system_code, source in self.sources.items():
            if system_code in self.pricing_engine.price_history:
                history = self.pricing_engine.price_history[system_code]
                
                # Convert timestamps to datetime objects
                timestamps = [pd.to_datetime(ts) for ts in history['timestamps']]
                
                # Update data source
                source.data = dict(x=timestamps, y=history['prices'])
    
    def create_comparison_plot(self):
        """Create comparison plot for all models"""
        p = figure(title="Pricing Model Comparison", 
                  x_axis_label='Time Steps', y_axis_label='Price ($)',
                  width=800, height=400)
        
        # This would show comparison between Model 1, 2, and 3
        # Implementation depends on stored historical data
        
        return p


In [13]:
def generate_analysis_report(pricing_engine: ParkingPriceEngine, df: pd.DataFrame):
    """Generate comprehensive analysis report"""
    print("\n" + "="*60)
    print("📋 FINAL ANALYSIS REPORT")
    print("="*60)
    
    # Price statistics
    print("\n💰 PRICING STATISTICS:")
    print("-" * 30)
    
    if pricing_engine.current_prices:
        prices = list(pricing_engine.current_prices.values())
        print(f"Average Price: ${np.mean(prices):.2f}")
        print(f"Price Range: ${min(prices):.2f} - ${max(prices):.2f}")
        print(f"Price Std Dev: ${np.std(prices):.2f}")
    
    # Occupancy analysis
    print("\n🅿️  OCCUPANCY ANALYSIS:")
    print("-" * 30)
    
    occupancy_stats = df.groupby('SystemCodeNumber').agg({
        'Occupancy': ['mean', 'max'],
        'Capacity': 'first'
    }).round(2)
    
    occupancy_stats.columns = ['Avg_Occupancy', 'Max_Occupancy', 'Capacity']
    occupancy_stats['Avg_Occupancy_Rate'] = (occupancy_stats['Avg_Occupancy'] / 
                                            occupancy_stats['Capacity'] * 100).round(1)
    
    print(occupancy_stats.head())
    
    # Traffic condition analysis
    print("\n🚦 TRAFFIC CONDITION IMPACT:")
    print("-" * 30)
    
    traffic_analysis = df.groupby('TrafficConditionNearby').agg({
        'QueueLength': 'mean',
        'Occupancy': 'mean'
    }).round(2)
    
    print(traffic_analysis)
    
    # Vehicle type analysis
    print("\n🚗 VEHICLE TYPE DISTRIBUTION:")
    print("-" * 30)
    
    vehicle_dist = df['VehicleType'].value_counts(normalize=True) * 100
    print(vehicle_dist.round(1))
    
    print("\n✅ Analysis complete!")


In [14]:
def main():
    """Main execution function"""
    print("🚗 Dynamic Pricing for Urban Parking Lots")
    print("=" * 50)
    
    # Initialize pricing engine
    pricing_engine = ParkingPriceEngine(base_price=10.0)
    
    # Load data
    print("📊 Loading dataset...")
    simulator = RealTimeSimulator('dataset.csv', pricing_engine)
    df = simulator.load_and_prepare_data()
    
    print(f"✅ Loaded {len(df)} records from {df['SystemCodeNumber'].nunique()} parking lots")
    print(f"📅 Date range: {df['DateTime'].min()} to {df['DateTime'].max()}")
    
    # Create visualization
    print("\n📈 Setting up real-time visualization...")
    viz_engine = VisualizationEngine(pricing_engine)
    system_codes = df['SystemCodeNumber'].unique()
    
    # Create plots
    grid = viz_engine.create_real_time_plots(system_codes)
    
    # Show initial plots
    show(grid)
    
    # Start simulation in a separate thread
    def run_simulation():
        simulator.simulate_real_time_stream(df.head(100), delay_seconds=0.5)  # Use first 100 records for demo
    
    simulation_thread = threading.Thread(target=run_simulation)
    simulation_thread.daemon = True
    simulation_thread.start()
    
    # Update plots periodically
    def update_visualization():
        for _ in range(50):  # Update 50 times
            time.sleep(1)
            viz_engine.update_plots()
    
    viz_thread = threading.Thread(target=update_visualization)
    viz_thread.daemon = True
    viz_thread.start()
    
    # Wait for simulation to complete
    simulation_thread.join()
    
    print("\n📊 Generating final analysis...")
    generate_analysis_report(pricing_engine, df)


In [15]:
# Run the complete dynamic pricing system
if __name__ == "__main__":
    main()


🚗 Dynamic Pricing for Urban Parking Lots
📊 Loading dataset...
✅ Loaded 18368 records from 14 parking lots
📅 Date range: 2016-10-04 07:59:00 to 2016-12-19 16:30:00

📈 Setting up real-time visualization...


Starting real-time simulation...

🕐 Timestamp: 2016-10-04 07:59:00
------------------------------------------------------------
🅿️  BHMBCCMKT01: Occupancy: 61/577 (10.6%) | Queue: 1 | Traffic: low | Price: $12.15
   💡 Price reduced due to competition
🅿️  BHMNCPHST01: Occupancy: 237/1200 (19.8%) | Queue: 2 | Traffic: low | Price: $12.44
   💡 Price reduced due to competition
🅿️  BHMMBMMBX01: Occupancy: 264/687 (38.4%) | Queue: 2 | Traffic: low | Price: $13.77
   💡 No nearby competitors
🅿️  BHMNCPNST01: Occupancy: 249/485 (51.3%) | Queue: 2 | Traffic: low | Price: $13.32
   💡 Price reduced due to competition
🅿️  Shopping: Occupancy: 614/1920 (32.0%) | Queue: 2 | Traffic: low | Price: $12.71
   💡 Price reduced due to competition
   ... and 9 more parking lots

🕐 Timestamp: 2016-10-04 08:25:00
------------------------------------------------------------
🅿️  Others-CCCPS8: Occupancy: 478/1322 (36.2%) | Queue: 2 | Traffic: low | Price: $13.88
🅿️  BHMNCPNST01: Occupancy: 267/485 (55.1%) | Queu

In [16]:
# Test individual pricing models
def test_models():
    """Test individual pricing models with sample data"""
    engine = ParkingPriceEngine()
    
    # Sample data
    system_code = "TEST01"
    occupancy = 150
    capacity = 200
    queue_length = 5
    traffic_condition = "high"
    is_special_day = 0
    vehicle_type = "car"
    
    # Test Model 1
    model1_price = engine.model1_baseline_linear(system_code, occupancy, capacity)
    print(f"Model 1 Price: ${model1_price:.2f}")
    
    # Test Model 2
    model2_price = engine.model2_demand_based(
        system_code, occupancy, capacity, queue_length, 
        traffic_condition, is_special_day, vehicle_type
    )
    print(f"Model 2 Price: ${model2_price:.2f}")
    
    # Test Model 3 (requires mock data)
    mock_parking_data = {
        "TEST01": {
            'occupancy': occupancy,
            'capacity': capacity,
            'latitude': 26.1445,
            'longitude': 91.7362
        },
        "TEST02": {
            'occupancy': 100,
            'capacity': 150,
            'latitude': 26.1400,
            'longitude': 91.7310
        }
    }
    
    model3_price, suggestion = engine.model3_competitive_pricing(
        system_code, occupancy, capacity, queue_length,
        traffic_condition, is_special_day, vehicle_type,
        26.1445, 91.7362, mock_parking_data
    )
    print(f"Model 3 Price: ${model3_price:.2f}")
    print(f"Suggestion: {suggestion}")

# Uncomment to test individual models
# test_models()


In [17]:


# class ParkingPriceEngine:
#     """
#     Dynamic Pricing Engine for Urban Parking Lots
#     Implements three pricing models with increasing complexity
#     """
    
#     def __init__(self, base_price: float = 10.0):
#         self.base_price = base_price
#         self.current_prices = {}
#         self.price_history = {}
#         self.demand_history = {}
        
#         # Model parameters
#         self.model1_alpha = 0.5  # Linear model coefficient
#         self.model2_params = {
#             'occupancy_weight': 2.0,
#             'queue_weight': 1.5,
#             'traffic_weight': 0.8,
#             'special_day_weight': 1.2,
#             'vehicle_type_weights': {'car': 1.0, 'bike': 0.7, 'truck': 1.3, 'cycle': 0.6}
#         }
        
#         # Price bounds
#         self.min_price_multiplier = 0.5
#         self.max_price_multiplier = 2.0
        
#     def calculate_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
#         """Calculate distance between two points using Haversine formula"""
#         R = 6371  # Earth's radius in kilometers
        
#         lat1_rad = math.radians(lat1)
#         lon1_rad = math.radians(lon1)
#         lat2_rad = math.radians(lat2)
#         lon2_rad = math.radians(lon2)
        
#         dlat = lat2_rad - lat1_rad
#         dlon = lon2_rad - lon1_rad
        
#         a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
#         c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        
#         return R * c
    
#     def model1_baseline_linear(self, system_code: str, occupancy: int, capacity: int, 
#                               previous_price: float = None) -> float:
#         """
#         Model 1: Baseline Linear Model
#         Price increases linearly with occupancy rate
#         """
#         if previous_price is None:
#             previous_price = self.base_price
            
#         occupancy_rate = occupancy / capacity if capacity > 0 else 0
#         price_adjustment = self.model1_alpha * occupancy_rate
        
#         new_price = previous_price + price_adjustment
        
#         # Apply bounds
#         min_price = self.base_price * self.min_price_multiplier
#         max_price = self.base_price * self.max_price_multiplier
        
#         return max(min_price, min(max_price, new_price))
    
#     def model2_demand_based(self, system_code: str, occupancy: int, capacity: int,
#                            queue_length: int, traffic_condition: str, is_special_day: int,
#                            vehicle_type: str) -> float:
#         """
#         Model 2: Demand-Based Price Function
#         Uses multiple factors to calculate demand and adjust price
#         """
#         params = self.model2_params
        
#         # Calculate occupancy rate
#         occupancy_rate = occupancy / capacity if capacity > 0 else 0
        
#         # Traffic condition mapping
#         traffic_multiplier = {'low': 0.5, 'average': 1.0, 'high': 1.5}.get(traffic_condition, 1.0)
        
#         # Vehicle type weight
#         vehicle_weight = params['vehicle_type_weights'].get(vehicle_type, 1.0)
        
#         # Calculate demand function
#         demand = (params['occupancy_weight'] * occupancy_rate +
#                  params['queue_weight'] * (queue_length / 10) +  # Normalize queue length
#                  params['traffic_weight'] * traffic_multiplier +
#                  params['special_day_weight'] * is_special_day +
#                  0.5 * vehicle_weight)
        
#         # Normalize demand (sigmoid function for smooth transitions)
#         normalized_demand = 2 / (1 + math.exp(-demand)) - 1  # Range: [-1, 1]
        
#         # Calculate price
#         price_multiplier = 1 + 0.5 * normalized_demand  # Range: [0.5, 1.5]
#         new_price = self.base_price * price_multiplier
        
#         # Apply bounds
#         min_price = self.base_price * self.min_price_multiplier
#         max_price = self.base_price * self.max_price_multiplier
        
#         return max(min_price, min(max_price, new_price))
    
    
#     def model3_competitive_pricing(self, system_code: str, occupancy: int, capacity: int,
#                                   queue_length: int, traffic_condition: str, is_special_day: int,
#                                   vehicle_type: str, latitude: float, longitude: float,
#                                   all_parking_ Dict) -> Tuple[float, str]:
#         """
#         Model 3: Competitive Pricing Model
#         Considers nearby parking lots and their prices for competitive pricing
#         """
#         # Start with demand-based price
#         base_price = self.model2_demand_based(system_code, occupancy, capacity, queue_length,
#                                             traffic_condition, is_special_day, vehicle_type)
        
#         # Find nearby competitors (within 2km)
#         nearby_lots = []
#         for other_code, other_data in all_parking_data.items():
#             if other_code != system_code:
#                 distance = self.calculate_distance(latitude, longitude,
#                                                  other_data['latitude'], other_data['longitude'])
#                 if distance <= 2.0:  # Within 2km
#                     nearby_lots.append({
#                         'code': other_code,
#                         'distance': distance,
#                         'occupancy_rate': other_data['occupancy'] / other_data['capacity'],
#                         'price': self.current_prices.get(other_code, self.base_price)
#                     })
        
#         if not nearby_lots:
#             return base_price, "No nearby competitors"
        
#         # Sort by distance
#         nearby_lots.sort(key=lambda x: x['distance'])
        
#         # Calculate competitive adjustment
#         current_occupancy_rate = occupancy / capacity if capacity > 0 else 0
        
#         # If current lot is nearly full (>90%), suggest rerouting to less occupied nearby lots
#         if current_occupancy_rate > 0.9:
#             available_alternatives = [lot for lot in nearby_lots if lot['occupancy_rate'] < 0.8]
#             if available_alternatives:
#                 cheapest_alternative = min(available_alternatives, key=lambda x: x['price'])
#                 suggestion = f"Consider rerouting to {cheapest_alternative['code']} (${cheapest_alternative['price']:.2f})"
#                 # Increase price to discourage more entries
#                 competitive_price = base_price * 1.2
#             else:
#                 suggestion = "All nearby lots are busy"
#                 competitive_price = base_price * 1.3
#         else:
#             # Normal competitive pricing
#             avg_nearby_price = sum(lot['price'] for lot in nearby_lots) / len(nearby_lots)
            
#             if base_price > avg_nearby_price * 1.1:
#                 # Our price is too high, reduce it
#                 competitive_price = base_price * 0.95
#                 suggestion = "Price reduced due to competition"
#             elif base_price < avg_nearby_price * 0.9:
#                 # We can increase price
#                 competitive_price = base_price * 1.05
#                 suggestion = "Price increased - competitive advantage"
#             else:
#                 competitive_price = base_price
#                 suggestion = "Price competitive with nearby lots"
        
#         # Apply bounds
#         min_price = self.base_price * self.min_price_multiplier
#         max_price = self.base_price * self.max_price_multiplier
#         final_price = max(min_price, min(max_price, competitive_price))
        
#         return final_price, suggestion
    
#     def update_price_history(self, system_code: str, price: float, timestamp: str):
#         """Update price history for visualization"""
#         if system_code not in self.price_history:
#             self.price_history[system_code] = {'timestamps': [], 'prices': []}
        
#         self.price_history[system_code]['timestamps'].append(timestamp)
#         self.price_history[system_code]['prices'].append(price)
        
#         # Keep only last 50 points for visualization
#         if len(self.price_history[system_code]['timestamps']) > 50:
#             self.price_history[system_code]['timestamps'] = self.price_history[system_code]['timestamps'][-50:]
#             self.price_history[system_code]['prices'] = self.price_history[system_code]['prices'][-50:]

# class RealTimeSimulator:
#     """
#     Real-time data simulation using Pathway
#     """
    
#     def __init__(self, data_file: str, pricing_engine: ParkingPriceEngine):
#         self.data_file = data_file
#         self.pricing_engine = pricing_engine
#         self.current_data = {}
        
#     def load_and_prepare_data(self):
#         """Load and prepare the dataset"""
#         df = pd.read_csv(self.data_file)
        
#         # Convert date and time columns
#         df['DateTime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], 
#                                        format='%d-%m-%Y %H:%M:%S')
        
#         # Sort by datetime
#         df = df.sort_values('DateTime')
        
#         return df
    
#     def simulate_real_time_stream(self, df: pd.DataFrame, delay_seconds: float = 1.0):
#         """
#         Simulate real-time data stream with specified delay
#         """
#         print("Starting real-time simulation...")
#         print("=" * 60)
        
#         # Group data by system code for easier processing
#         grouped_data = {}
#         for system_code in df['SystemCodeNumber'].unique():
#             system_data = df[df['SystemCodeNumber'] == system_code].copy()
#             grouped_data[system_code] = system_data
        
#         # Get all unique timestamps
#         all_timestamps = sorted(df['DateTime'].unique())
        
#         for timestamp in all_timestamps:
#             current_batch = df[df['DateTime'] == timestamp]
            
#             # Update current data state
#             all_parking_data = {}
#             for _, row in current_batch.iterrows():
#                 system_code = row['SystemCodeNumber']
#                 all_parking_data[system_code] = {
#                     'occupancy': row['Occupancy'],
#                     'capacity': row['Capacity'],
#                     'latitude': row['Latitude'],
#                     'longitude': row['Longitude'],
#                     'queue_length': row['QueueLength'],
#                     'traffic_condition': row['TrafficConditionNearby'],
#                     'is_special_day': row['IsSpecialDay'],
#                     'vehicle_type': row['VehicleType']
#                 }
            
#             # Process each parking lot
#             results = []
#             for system_code, data in all_parking_data.items():
#                 # Get previous price
#                 previous_price = self.pricing_engine.current_prices.get(system_code, 
#                                                                        self.pricing_engine.base_price)
                
#                 # Calculate prices using all three models
#                 model1_price = self.pricing_engine.model1_baseline_linear(
#                     system_code, data['occupancy'], data['capacity'], previous_price
#                 )
                
#                 model2_price = self.pricing_engine.model2_demand_based(
#                     system_code, data['occupancy'], data['capacity'], data['queue_length'],
#                     data['traffic_condition'], data['is_special_day'], data['vehicle_type']
#                 )
                
#                 model3_price, suggestion = self.pricing_engine.model3_competitive_pricing(
#                     system_code, data['occupancy'], data['capacity'], data['queue_length'],
#                     data['traffic_condition'], data['is_special_day'], data['vehicle_type'],
#                     data['latitude'], data['longitude'], all_parking_data
#                 )
                
#                 # Update current price (using Model 3 as final price)
#                 self.pricing_engine.current_prices[system_code] = model3_price
                
#                 # Update price history
#                 self.pricing_engine.update_price_history(system_code, model3_price, str(timestamp))
                
#                 results.append({
#                     'timestamp': timestamp,
#                     'system_code': system_code,
#                     'occupancy': data['occupancy'],
#                     'capacity': data['capacity'],
#                     'occupancy_rate': data['occupancy'] / data['capacity'],
#                     'queue_length': data['queue_length'],
#                     'traffic': data['traffic_condition'],
#                     'special_day': data['is_special_day'],
#                     'vehicle_type': data['vehicle_type'],
#                     'model1_price': model1_price,
#                     'model2_price': model2_price,
#                     'model3_price': model3_price,
#                     'suggestion': suggestion
#                 })
            
#             # Display current status
#             self.display_current_status(timestamp, results)
            
#             # Wait before next update
#             time.sleep(delay_seconds)
            
#         print("\nSimulation completed!")
    
#     def display_current_status(self, timestamp, results):
#         """Display current pricing status"""
#         print(f"\n🕐 Timestamp: {timestamp}")
#         print("-" * 60)
        
#         for result in results[:5]:  # Show first 5 for brevity
#             occupancy_rate = result['occupancy_rate'] * 100
#             print(f"🅿️  {result['system_code']}: "
#                   f"Occupancy: {result['occupancy']}/{result['capacity']} ({occupancy_rate:.1f}%) | "
#                   f"Queue: {result['queue_length']} | "
#                   f"Traffic: {result['traffic']} | "
#                   f"Price: ${result['model3_price']:.2f}")
            
#             if result['suggestion'] != "Price competitive with nearby lots":
#                 print(f"   💡 {result['suggestion']}")
        
#         if len(results) > 5:
#             print(f"   ... and {len(results) - 5} more parking lots")

# class VisualizationEngine:
#     """
#     Real-time visualization using Bokeh
#     """
    
#     def __init__(self, pricing_engine: ParkingPriceEngine):
#         self.pricing_engine = pricing_engine
#         self.plots = {}
#         self.sources = {}
    
#     def create_real_time_plots(self, system_codes: List[str]):
#         """Create real-time pricing plots"""
#         plots = []
        
#         for i, system_code in enumerate(system_codes[:6]):  # Show first 6 plots
#             # Create data source
#             source = ColumnDataSource(data=dict(x=[], y=[]))
#             self.sources[system_code] = source
            
#             # Create plot
#             p = figure(title=f"Real-time Pricing - {system_code}", 
#                       x_axis_label='Time', y_axis_label='Price ($)',
#                       width=400, height=300,
#                       x_axis_type='datetime')
            
#             p.line('x', 'y', source=source, line_width=2, color='blue')
#             p.circle('x', 'y', source=source, size=6, color='red')
            
#             plots.append(p)
#             self.plots[system_code] = p
        
#         # Create grid layout
#         grid = gridplot([plots[i:i+3] for i in range(0, len(plots), 3)])
        
#         return grid
    
#     def update_plots(self):
#         """Update plots with latest data"""
#         for system_code, source in self.sources.items():
#             if system_code in self.pricing_engine.price_history:
#                 history = self.pricing_engine.price_history[system_code]
                
#                 # Convert timestamps to datetime objects
#                 timestamps = [pd.to_datetime(ts) for ts in history['timestamps']]
                
#                 # Update data source
#                 source.data = dict(x=timestamps, y=history['prices'])

# # Main execution
# def main():
#     """Main execution function"""
#     print("🚗 Dynamic Pricing for Urban Parking Lots")
#     print("=" * 50)
    
#     # Initialize pricing engine
#     pricing_engine = ParkingPriceEngine(base_price=10.0)
    
#     # Load data
#     print("📊 Loading dataset...")
#     simulator = RealTimeSimulator('dataset.csv', pricing_engine)
#     df = simulator.load_and_prepare_data()
    
#     print(f"✅ Loaded {len(df)} records from {df['SystemCodeNumber'].nunique()} parking lots")
#     print(f"📅 Date range: {df['DateTime'].min()} to {df['DateTime'].max()}")
    
#     # Create visualization
#     print("\n📈 Setting up real-time visualization...")
#     viz_engine = VisualizationEngine(pricing_engine)
#     system_codes = df['SystemCodeNumber'].unique()
    
#     # Create plots
#     grid = viz_engine.create_real_time_plots(system_codes)
    
#     # Show initial plots
#     show(grid)
    
#     # Start simulation in a separate thread
#     def run_simulation():
#         simulator.simulate_real_time_stream(df.head(100), delay_seconds=0.5)  # Use first 100 records for demo
    
#     simulation_thread = threading.Thread(target=run_simulation)
#     simulation_thread.daemon = True
#     simulation_thread.start()
    
#     # Update plots periodically
#     def update_visualization():
#         for _ in range(50):  # Update 50 times
#             time.sleep(1)
#             viz_engine.update_plots()
    
#     viz_thread = threading.Thread(target=update_visualization)
#     viz_thread.daemon = True
#     viz_thread.start()
    
#     # Wait for simulation to complete
#     simulation_thread.join()
    
#     print("\n📊 Generating final analysis...")
#     generate_analysis_report(pricing_engine, df)

# def generate_analysis_report(pricing_engine: ParkingPriceEngine, df: pd.DataFrame):
#     """Generate comprehensive analysis report"""
#     print("\n" + "="*60)
#     print("📋 FINAL ANALYSIS REPORT")
#     print("="*60)
    
#     # Price statistics
#     print("\n💰 PRICING STATISTICS:")
#     print("-" * 30)
    
#     if pricing_engine.current_prices:
#         prices = list(pricing_engine.current_prices.values())
#         print(f"Average Price: ${np.mean(prices):.2f}")
#         print(f"Price Range: ${min(prices):.2f} - ${max(prices):.2f}")
#         print(f"Price Std Dev: ${np.std(prices):.2f}")
    
#     # Occupancy analysis
#     print("\n🅿️  OCCUPANCY ANALYSIS:")
#     print("-" * 30)
    
#     occupancy_stats = df.groupby('SystemCodeNumber').agg({
#         'Occupancy': ['mean', 'max'],
#         'Capacity': 'first'
#     }).round(2)
    
#     occupancy_stats.columns = ['Avg_Occupancy', 'Max_Occupancy', 'Capacity']
#     occupancy_stats['Avg_Occupancy_Rate'] = (occupancy_stats['Avg_Occupancy'] / 
#                                             occupancy_stats['Capacity'] * 100).round(1)
    
#     print(occupancy_stats.head())
    
#     # Traffic condition analysis
#     print("\n🚦 TRAFFIC CONDITION IMPACT:")
#     print("-" * 30)
    
#     traffic_analysis = df.groupby('TrafficConditionNearby').agg({
#         'QueueLength': 'mean',
#         'Occupancy': 'mean'
#     }).round(2)
    
#     print(traffic_analysis)
    
#     # Vehicle type analysis
#     print("\n🚗 VEHICLE TYPE DISTRIBUTION:")
#     print("-" * 30)
    
#     vehicle_dist = df['VehicleType'].value_counts(normalize=True) * 100
#     print(vehicle_dist.round(1))
    
#     print("\n✅ Analysis complete!")

# # Run the complete solution
# if __name__ == "__main__":
#     main()
